In [8]:
import pandas as pd
import re

In [31]:
df = pd.read_csv('~/project/MIMIC-III/Data Extract/Tabular Data/FINAL Data Extract/DATA/Demo+Lab+Chart/FINAL_Demo_Lab_Chart(100%).csv')
cat = pd.read_csv('~/project/MIMIC-III/Data Extract/Tabular Data/FINAL Data Extract/DATA/FINAL_cat.csv')
cat.drop(['y'], axis=1, inplace=True)

In [32]:
df1 = pd.merge(df, cat, on=['SUBJECT_ID', 'HADM_ID'], how='left')
df1

,SUBJECT_ID,HADM_ID,TLOS,AGE,DOA,ETHNICITY,GENDER,LOS,Creatinine_mean,Urea Nitrogen_mean,...,SpO2_max,SpO2_min,Ventilator,Anisocytosis,Macrocytes,Poikilocytosis,Bacteria,Bilirubin,Urine Appearance,Urine Color
0,101,175533,15 days 18:41:00,82,15 days 18:41:00,ASIAN,M,9.8919,0.925000,16.000000,...,100.0,67.0,CONTROLLED,NaN,NaN,NaN,OTHER,NEG,AMBER,YELLOW
1,111,192123,11 days 04:50:00,66,NaN,WHITE,F,10.5708,0.600000,12.500000,...,100.0,92.0,NaN,NORMAL,2+,NORMAL,MOD,NEG,CLOUDY,YELLOW
2,141,168006,3 days 09:29:00,80,3 days 09:29:00,OTHER,M,3.4765,1.466667,29.166667,...,100.0,0.0,CONTROLLED,NORMAL,NORMAL,NORMAL,OTHER,NEG,AMBER,YELLOW
3,148,199488,32 days 22:37:00,78,NaN,UNKNOWN/NOT SPECIFIED,F,15.2614,1.000000,23.000000,...,100.0,0.0,NaN,1+,1+,1+,OTHER,NEG,CLEAR,STRAW
4,170,175350,4 days 00:00:00,71,NaN,WHITE,F,3.7791,0.700000,14.000000,...,100.0,90.0,NaN,NaN,NaN,NaN,OTHER,NaN,NaN,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,32763,160695,33 days 20:31:00,79,33 days 20:31:00,WHITE,M,31.0886,1.400000,35.000000,...,100.0,90.0,NaN,NORMAL,NORMAL,1+,OTHER,NEG,CLEAR,YELLOW
2278,32779,133757,6 days 14:39:00,69,6 days 14:39:00,WHITE,F,7.1648,1.466667,30.333333,...,100.0,90.0,CONTROLLED,NORMAL,NORMAL,NORMAL,OTHER,NEG,CLEAR,YELLOW
2279,32781,142684,22 days 14:41:00,86,NaN,WHITE,M,9.8333,2.025000,29.250000,...,100.0,88.0,NaN,1+,NORMAL,1+,OTHER,LG,AMBER,OTHER
2280,32786,165351,22 days 14:34:00,53,NaN,WHITE,M,22.6358,1.033333,15.666667,...,100.0,87.0,NaN,NaN,NaN,NaN,OTHER,NEG,CLEAR,YELLOW


In [33]:
# 'TLOS' 열의 데이터를 파싱하여 일(day), 시간(hour), 분(minute) 정보 추출
def parse_tlos(row):
    match = re.search(r'(\d+) days (\d+):(\d+):(\d+)', row['TLOS'])
    if match:
        days, hours, minutes, seconds = map(int, match.groups())
        return pd.Timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)
    return pd.NaT

df1['TLOS'] = df1.apply(parse_tlos, axis=1)
df1['TLOS'] = df1['TLOS'].dt.days + ((df1['TLOS'].dt.seconds / 3600) / 24)
df1

,SUBJECT_ID,HADM_ID,TLOS,AGE,DOA,ETHNICITY,GENDER,LOS,Creatinine_mean,Urea Nitrogen_mean,...,SpO2_max,SpO2_min,Ventilator,Anisocytosis,Macrocytes,Poikilocytosis,Bacteria,Bilirubin,Urine Appearance,Urine Color
0,101,175533,15.778472,82,15 days 18:41:00,ASIAN,M,9.8919,0.925000,16.000000,...,100.0,67.0,CONTROLLED,NaN,NaN,NaN,OTHER,NEG,AMBER,YELLOW
1,111,192123,11.201389,66,NaN,WHITE,F,10.5708,0.600000,12.500000,...,100.0,92.0,NaN,NORMAL,2+,NORMAL,MOD,NEG,CLOUDY,YELLOW
2,141,168006,3.395139,80,3 days 09:29:00,OTHER,M,3.4765,1.466667,29.166667,...,100.0,0.0,CONTROLLED,NORMAL,NORMAL,NORMAL,OTHER,NEG,AMBER,YELLOW
3,148,199488,32.942361,78,NaN,UNKNOWN/NOT SPECIFIED,F,15.2614,1.000000,23.000000,...,100.0,0.0,NaN,1+,1+,1+,OTHER,NEG,CLEAR,STRAW
4,170,175350,4.000000,71,NaN,WHITE,F,3.7791,0.700000,14.000000,...,100.0,90.0,NaN,NaN,NaN,NaN,OTHER,NaN,NaN,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,32763,160695,33.854861,79,33 days 20:31:00,WHITE,M,31.0886,1.400000,35.000000,...,100.0,90.0,NaN,NORMAL,NORMAL,1+,OTHER,NEG,CLEAR,YELLOW
2278,32779,133757,6.610417,69,6 days 14:39:00,WHITE,F,7.1648,1.466667,30.333333,...,100.0,90.0,CONTROLLED,NORMAL,NORMAL,NORMAL,OTHER,NEG,CLEAR,YELLOW
2279,32781,142684,22.611806,86,NaN,WHITE,M,9.8333,2.025000,29.250000,...,100.0,88.0,NaN,1+,NORMAL,1+,OTHER,LG,AMBER,OTHER
2280,32786,165351,22.606944,53,NaN,WHITE,M,22.6358,1.033333,15.666667,...,100.0,87.0,NaN,NaN,NaN,NaN,OTHER,NEG,CLEAR,YELLOW


In [34]:
df1.to_csv("~/project/MIMIC-III/Data Extract/Tabular Data/FINAL Data Extract/DATA/Demo+Lab+Chart/FINAL_Demo_Lab_Chart(100%).csv", index=False)